In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# DataFrame 만들기
  * products.csv
  * services.csv
  * final.csv

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
csv_lists = os.listdir('/content/drive/MyDrive/Project/lotte/DATA')
csv_lists = csv_lists[2:]

In [ ]:
csv_lists

In [ ]:
demo = pd.read_csv('./data/'+csv_lists[0]) # demo 와 coporation에 merge 됨
buy = pd.read_csv('./data/'+csv_lists[1])
coporation= pd.read_csv('./data/'+csv_lists[2])  # 제휴사
partner = pd.read_csv('./data/'+csv_lists[3])  # buy 에 merge 됨  # 유통사
stores= pd.read_csv('./data/'+csv_lists[4])   # 점포
Lpay = pd.read_csv('./data/'+csv_lists[5])  # Lpay

In [ ]:
demo.columns = ['고객번호','성별', '연령대', '거주지대분류코드']
buy.columns = ['고객번호', '영수증번호', '채널구분(1:오프, 2:온)', '제휴사(유통사만 A01~06)', '점포코드', '상품코드', '구매일자', '구매시간', '구매금액' , '구매수량']
coporation.columns = ['고객번호', '영수증번호', '제휴사(B~E:숙박/엔터/F&B/렌탈)', '점포코드', '채널구분(1:오프, 2:온)', '이용일자', '방문일자', '이용시간', '이용금액']
partner.columns = ['상품코드', '소분류명', '중분류명', '대분류명']
stores.columns = ['점포코드', '제휴사', '점포지역대분류', '점포지역중분류']
Lpay.columns = ['고객번호', '영수증번호', '제휴사', '채널구분(1:오프, 2:온)', '이용일자', '이용시간', '이용금액']

## products DataFrame 만들기

In [ ]:
buy

In [ ]:
partner

In [ ]:
buy

In [ ]:
product = pd.merge(buy, partner, on = '상품코드', how = 'left')

In [ ]:
product_1 = product[product['채널구분(1:오프, 2:온)']==1]
product_2 = product[product['채널구분(1:오프, 2:온)']==2]

In [ ]:
len(product_1)

In [ ]:
stores_product = stores[(stores['제휴사']=='A01') | (stores['제휴사']=='A02') | (stores['제휴사']=='A03') | (stores['제휴사']=='A04') | (stores['제휴사']=='A05') | (stores['제휴사']=='A06')]

In [ ]:
product_1 = pd.merge(product_1, stores_product, on = '점포코드', how = 'left')
len(product_1)

In [ ]:
product_1 = product_1[['고객번호', '영수증번호', '채널구분(1:오프, 2:온)', '제휴사(유통사만 A01~06)', '점포코드', '상품코드',
       '구매일자', '구매시간', '구매금액', '구매수량', '소분류명', '중분류명', '대분류명',
       '점포지역대분류', '점포지역중분류']]

In [ ]:
product_2['점포지역대분류'] = np.nan
product_2['점포지역중분류'] = np.nan

In [ ]:
product = pd.concat([product_1, product_2])

In [ ]:
product = pd.merge(product, demo, on = '고객번호', how = 'left')
product = product.dropna(subset=['영수증번호'])

In [ ]:
Lpay_product = Lpay[(Lpay['제휴사']=='A01') | (Lpay['제휴사']=='A02') | (Lpay['제휴사']=='A03') | (Lpay['제휴사']=='A04') | (Lpay['제휴사']=='A05') | (Lpay['제휴사']=='A06')]
Lpay_product.columns = ['고객번호', '영수증번호', '제휴사(유통사만 A01~06)', '채널구분(1:오프, 2:온)', '구매일자', '구매시간', '구매금액']
Lpay_product['구매금액'] = Lpay_product['구매금액'].astype('float64')
Lpay_product = Lpay_product[['고객번호', '제휴사(유통사만 A01~06)', '채널구분(1:오프, 2:온)', '구매일자', '구매시간', '구매금액']]
Lpay_product                            

In [ ]:
Lpay_client = Lpay_product['고객번호'].value_counts().index.tolist()

In [ ]:
len(Lpay_client)

In [ ]:
product = product.reset_index(drop = True)

In [ ]:
product['Lpay_client']=0
for i in tqdm(range(len(Lpay_client))) : 
    product.iloc[product[product['고객번호'] == Lpay_client[i]].index,-1]=1

In [ ]:
product = product[['영수증번호', '고객번호', '성별', '연령대', '거주지대분류코드',  '상품코드', '대분류명', '중분류명', '소분류명', 
       '구매일자', '구매시간',  '구매수량', '구매금액',  '제휴사(유통사만 A01~06)','점포코드', '점포지역대분류',
       '점포지역중분류',  '채널구분(1:오프, 2:온)', 'Lpay_client']]

In [ ]:
product

In [ ]:
product['채널구분(1:오프, 2:온)'] = product['채널구분(1:오프, 2:온)'].astype(int)
Lpay_product['채널구분(1:오프, 2:온)'] = Lpay_product['채널구분(1:오프, 2:온)'].astype(int)

product['구매일자'] = product['구매일자'].astype(int)
Lpay_product['구매일자'] = Lpay_product['구매일자'].astype(int)

product['구매시간'] = product['구매시간'].astype(int)
Lpay_product['구매시간'] = Lpay_product['구매시간'].astype(int)

product['구매금액'] = product['구매금액'].astype(float)
Lpay_product['구매금액'] = Lpay_product['구매금액'].astype(float)

In [ ]:
product.info()

In [ ]:
Lpay_product.info()

In [ ]:
product_Lpay = product[product['Lpay_client']==1]
product_n_Lpay = product[product['Lpay_client']==0]

In [ ]:
len(product_Lpay)

In [ ]:
product_Lpay['Lpay'] = 0
product_n_Lpay['Lpay'] = 0

In [ ]:
Lpay_index = []
for k in tqdm(range(len(Lpay_client))) : 
    tt = product_Lpay[product_Lpay['고객번호']==Lpay_client[k]][['고객번호', '제휴사(유통사만 A01~06)', '채널구분(1:오프, 2:온)', '구매일자', '구매시간', '구매금액']]
    qq = Lpay_product[Lpay_product['고객번호']==Lpay_client[k]]

    qq_list = []
    for i in range(len(qq)) : 
        qq_list.append(qq.iloc[i,:].tolist())
    for j in range(len(tt)) : 
        if tt.iloc[j,:].tolist() in qq_list : 
            Lpay_index.append(tt.index[j])

In [ ]:
product_Lpay.loc[Lpay_index, 'Lpay'] = 1

In [ ]:
product_Lpay['Lpay'].value_counts()

In [ ]:
product = pd.concat([product_Lpay, product_n_Lpay], axis = 0)
product = product.sort_values(['구매일자', '구매시간'])
product = product.reset_index(drop = True)

In [ ]:
product

In [ ]:
product.to_csv('./data/products.csv', index = False)

In [ ]:
pd.read_csv('./data/products.csv')

## services DataFrame 만들기

In [ ]:
service = pd.merge(coporation, demo, on = '고객번호', how = 'left')
service = service.dropna(subset=['영수증번호'])

In [ ]:
service_1 = service[service['채널구분(1:오프, 2:온)']==1.0]
service_2 = service[service['채널구분(1:오프, 2:온)']==2.0]

In [ ]:
len(service_1)

In [ ]:
service_1 = pd.merge(service_1, stores, on = '점포코드', how = 'left')
service_1 = service_1.dropna(subset=['고객번호'])
len(service_1)

In [ ]:
service = pd.concat([service_1, service_2], axis= 0)

In [ ]:
service = service.reset_index(drop = True)

In [ ]:
Lpay_product_index = Lpay[(Lpay['제휴사']=='A01') | (Lpay['제휴사']=='A02') | (Lpay['제휴사']=='A03') | (Lpay['제휴사']=='A04') | (Lpay['제휴사']=='A05') | (Lpay['제휴사']=='A06')].index
Lpay_service = Lpay.drop(Lpay_product_index)
Lpay_service.columns = ['고객번호', '영수증번호', '제휴사(B~E:숙박/엔터/F&B/렌탈)', '채널구분(1:오프, 2:온)', '이용일자', '이용시간', '이용금액']
Lpay_service['이용금액'] = Lpay_service['이용금액'].astype('float64')
Lpay_service = Lpay_service[['고객번호', '제휴사(B~E:숙박/엔터/F&B/렌탈)', '채널구분(1:오프, 2:온)', '이용일자', '이용시간', '이용금액']]
Lpay_service  

In [ ]:
Lpay_service_client = Lpay_service['고객번호'].value_counts().index.tolist()
len(Lpay_service_client)

In [ ]:
service['Lpay_client'] = 0
for i in tqdm(range(len(Lpay_service_client))) : 
    service.iloc[service[service['고객번호'] == Lpay_service_client[i]].index,-1]=1

In [ ]:
service = service[['영수증번호', '고객번호', '성별', '연령대', '거주지대분류코드', 
       '방문일자', '이용일자', '이용시간', '이용금액', '제휴사(B~E:숙박/엔터/F&B/렌탈)', '점포코드', '점포지역대분류', '점포지역중분류',
        '채널구분(1:오프, 2:온)','Lpay_client']]

In [ ]:
service

In [ ]:
service['채널구분(1:오프, 2:온)'] = service['채널구분(1:오프, 2:온)'].astype(int)
Lpay_service['채널구분(1:오프, 2:온)'] = Lpay_service['채널구분(1:오프, 2:온)'].astype(int)

service['이용일자'] = service['이용일자'].astype(int)
Lpay_service['이용일자'] = Lpay_service['이용일자'].astype(int)

service['이용시간'] = service['이용시간'].astype(int)
Lpay_service['이용시간'] = Lpay_service['이용시간'].astype(int)

In [ ]:
service.info()

In [ ]:
Lpay_service.info()

In [ ]:
service_Lpay = service[service['Lpay_client']==1]
service_n_Lpay = service[service['Lpay_client']==0]

In [ ]:
len(service_Lpay)

In [ ]:
service_Lpay['Lpay'] = 0
service_n_Lpay['Lpay'] = 0

In [ ]:
service_Lpay_index = []
for k in tqdm(range(len(Lpay_service_client))) : 
    tt = service_Lpay[service_Lpay['고객번호']==Lpay_service_client[k]][['고객번호', '제휴사(B~E:숙박/엔터/F&B/렌탈)', '채널구분(1:오프, 2:온)', '이용일자', '이용시간', '이용금액']]
    qq = Lpay_service[Lpay_service['고객번호']==Lpay_service_client[k]]

    qq_list = []
    for i in range(len(qq)) : 
        qq_list.append(qq.iloc[i,:].tolist())
    for j in range(len(tt)) : 
        if tt.iloc[j,:].tolist() in qq_list : 
            service_Lpay_index.append(tt.index[j])

In [ ]:
service_Lpay.loc[service_Lpay_index, 'Lpay'] = 1

In [ ]:
service_Lpay['Lpay'].value_counts()

In [ ]:
service = pd.concat([service_Lpay, service_n_Lpay], axis = 0)
service = service.sort_values(['이용일자', '이용시간'])
service = service.reset_index(drop = True)

In [ ]:
service

In [ ]:
service.to_csv('./data/services.csv',index = False)

In [ ]:
pd.read_csv('./data/services.csv')

## final DataFrame 만들기

In [ ]:
product = pd.read_csv('./data/products.csv')
service = pd.read_csv('./data/services.csv')

In [ ]:
product = product[['고객번호','성별','연령대','거주지대분류코드','구매일자','구매시간','구매금액','제휴사(유통사만 A01~06)','점포코드','점포지역대분류','채널구분(1:오프, 2:온)','Lpay']]
service = service[['고객번호','성별','연령대','거주지대분류코드','이용일자','이용시간','이용금액','제휴사(B~E:숙박/엔터/F&B/렌탈)','점포코드','점포지역대분류','채널구분(1:오프, 2:온)','Lpay']]

In [ ]:
product.rename(columns={'제휴사(유통사만 A01~06)':'제휴사'},inplace=True)
service.rename(columns={'이용일자':'구매일자','이용시간':'구매시간','이용금액':'구매금액','제휴사(B~E:숙박/엔터/F&B/렌탈)':'제휴사'},inplace=True)

In [ ]:
data = pd.concat([product,service],axis=0)
data = data.reset_index()
data.drop('index',axis=1,inplace=True)

In [ ]:
data['temp'] = data.groupby(['고객번호'])['Lpay'].transform('sum') 
data['Lpay_client'] = data['temp'].apply(lambda x: 1 if x > 0 else 0)  
data.drop('temp', axis=1, inplace=True)
data

In [ ]:
data.to_csv('./data/final.csv')

# 고객 생애 가치(Customer Lifetime Value:CLTV) 추정

In [ ]:
!pip install lifetimes
!pip install sqlalchemy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='Malgun Gothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="Malgun Gothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

product = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/final.csv')
product

In [ ]:
# 작성날짜 datetime으로 변환
product['구매일자'] = product['구매일자'].astype('str')
product['구매일자'] = pd.to_datetime(product['구매일자'])

In [ ]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
from lifetimes.utils import summary_data_from_transaction_data

summary = summary_data_from_transaction_data(product, '고객번호', '구매일자','구매금액', freq='D', observation_period_end='2021-12-31')
summary.head()

In [ ]:
len(summary)

In [ ]:
summary.describe()

In [ ]:
# BG-NBD 모델 생성 
bgf = BetaGeoFitter(penalizer_coef=0.001) #  0.001에서 0.1 정도가 일반적!
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

In [ ]:
from lifetimes.plotting import plot_frequency_recency_matrix

plot_frequency_recency_matrix(bgf)

In [ ]:
# 모델 적합성 평가
from lifetimes.plotting import plot_period_transactions

plot_period_transactions(bgf)
plt.show()

In [ ]:
# 데이터셋 분할
# 새로운 데이터에 대해 어떻게 작동하는지 테스트하기 위해

from lifetimes.utils import calibration_and_holdout_data

summary_cal_holdout = calibration_and_holdout_data(product, '고객번호', '구매일자',
                                        calibration_period_end='2021-09-01',
                                        observation_period_end='2021-12-31' )   
summary_cal_holdout.head()

In [ ]:
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases

bgf.fit(summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout)

In [ ]:
# 고객 예측
t = 10 #predict purchases in 10 periods
individual = summary.iloc[20]
# The below function is an alias to `bfg.conditional_expected_number_of_purchases_up_to_time`
bgf.predict(t, individual['frequency'], individual['recency'], individual['T'])

In [ ]:
# 고객의 재방문 확률 시계열
from lifetimes.plotting import plot_history_alive

id = 'M000136117' ## 고객 한명당 확률 보여줌
days_since_birth = 200
sp_trans = product.loc[product['고객번호'] == id]
plot_history_alive(bgf, days_since_birth, sp_trans, '구매일자')

GAMMA-GAMMA 모델 생성 & 고객의 평생 가치(CLV) 추정

In [ ]:
summary

In [ ]:
summary = summary[summary['frequency']>0]
summary

In [ ]:
summary[['monetary_value', 'frequency']].corr()

In [ ]:
from lifetimes import GammaGammaFitter

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(summary['frequency'],
        summary['monetary_value'])
print(ggf)

In [ ]:
print(ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']
    ).head(10))

In [ ]:
print("Expected conditional average profit: %s, Average profit: %s" % (
    ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']
    ).mean(),
    summary[summary['frequency']>0]['monetary_value'].mean()
))

In [ ]:
# refit the BG model to the summary_with_money_value dataset
#bgf.fit(summary['frequency'], summary['recency'], summary['T'])

print(ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    summary['frequency'],
    summary['recency'],
    summary['T'],
    summary['monetary_value'],
    time=7, # days
    freq='D',
    discount_rate=0.01 # monthly discount rate ~ 12.7% annually
).head(10))

In [ ]:
cltv = ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    summary['frequency'],
    summary['recency'],
    summary['T'],
    summary['monetary_value'],
    time=7, # days
    freq='D',
    discount_rate=0.01) # monthly discount rate ~ 12.7% annually
cltv = cltv.reset_index()
cltv.head()

In [ ]:
cltv_final = summary.merge(cltv, on="고객번호", how="left")

cltv_final

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1)) 
#score between 0-1 ,1 is best 0 is worst. You can change by your wish, if you want you can score between 0-100 .
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])
cltv_final.sort_values(by=["scaled_clv"], ascending=False).head()

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
cltv_final = cltv_final.sort_values(by=["clv"], ascending=False)
cltv_final

In [ ]:
cltv_final.to_csv('/content/gdrive/MyDrive/Lpoint_Lpay/data/final_cltv.csv')

In [ ]:
cltv_final['scaled_clv'].describe()

# 고객군 군집화

In [ ]:
#라이브러리 로드
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings(action='ignore')
import requests 
import json 
import xmltodict
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
%cd /Users/diekim/Desktop/ETC/LPOINT

In [ ]:
#함수 정의
### 실루엣 계수 시각화 함수: 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
def visualize_silhouette(cluster_lists, X_features): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
        
### 군집별 시각화 함수: 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 클러스터링 결과를 시각화 
def visualize_kmeans_plot_multi(cluster_lists, X_features):
    
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    import pandas as pd
    import numpy as np
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성 
    n_cols = len(cluster_lists)
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 입력 데이터의 FEATURE가 여러개일 경우 2차원 데이터 시각화가 어려우므로 PCA 변환하여 2차원 시각화
    pca = PCA(n_components=2)
    pca_transformed = pca.fit_transform(X_features)
    dataframe = pd.DataFrame(pca_transformed, columns=['PCA1','PCA2'])
    
     # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 KMeans 클러스터링 수행하고 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링으로 클러스터링 결과를 dataframe에 저장. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(pca_transformed)
        dataframe['cluster']=cluster_labels
        
        unique_labels = np.unique(clusterer.labels_)
        markers=['o', 's', '^', 'x', '*','d','+']
       
        # 클러스터링 결과값 별로 scatter plot 으로 시각화
        for label in unique_labels:
            label_df = dataframe[dataframe['cluster']==label]
            if label == -1:
                cluster_legend = 'Noise'
            else :
                cluster_legend = 'Cluster '+str(label)           
            axs[ind].scatter(x=label_df['PCA1'], y=label_df['PCA2'], s=70,\
                        edgecolor='k', marker=markers[label], label=cluster_legend)

        axs[ind].set_title('Number of Cluster : '+ str(n_cluster))    
        axs[ind].legend(loc='upper right')
    
    plt.show()

In [ ]:
#데이터 정리(only 재방문 유저)
final = pd.read_csv('final.csv').iloc[:, 1:]
cltv = pd.read_csv('final_cltv.csv').iloc[:, 1:]

df = pd.merge(final, cltv, on='고객번호', how='left')
df = df.dropna(subset=['frequency'])

df['구매일자'] = df['구매일자'].astype('str')
df['구매일자'] = pd.to_datetime(df['구매일자'])

df['d_min'] = df.groupby('고객번호')['구매일자'].transform('min')
df['d_max'] = df.groupby('고객번호')['구매일자'].transform('max')

df['R'] = df['d_max'] - df['d_min']
df['R'] = df['R'].apply(lambda x: x.days)

def m1(x):
    if x['Lpay']==1:
        return x['구매금액']
    else:
        return 0

def m2(x):
    if x['Lpay']==0:
        return x['구매금액']
    else:
        return 0
    
df['M1'] = df.apply(m1, axis=1)
df['M2'] = df.apply(m2, axis=1) 

client_df = df.groupby('고객번호').agg({'R':'mean',
                                      #'구매일자': 'count',
                                      '구매금액': 'mean',
                                      'M1': 'mean',
                                      'M2': 'mean',
                                      'Lpay': 'sum',
                                      'Lpay_client': 'mean'}).reset_index()

client_df = pd.merge(client_df, 
                     df[['고객번호', 'frequency', '성별', '연령대', '거주지대분류코드']],
                     on='고객번호', 
                     how='left').drop_duplicates().reset_index(drop=True)

client_df.columns = ['고객번호', 'R', 'M', 'M1', 'M2', 'Lpay', 'Lpay_client', 'F', '성별', '연령대', '거주지']

temp = df.groupby(['고객번호', '제휴사'])['구매금액'].sum().reset_index()
temp2 = temp.groupby('고객번호')['구매금액'].max().reset_index()
temp3 = pd.merge(temp, temp2, on=['고객번호', '구매금액'], how='inner')

client_df = pd.merge(client_df, temp3[['고객번호', '제휴사']], on='고객번호')
client_df.to_csv('final_client_df.csv', encoding='utf-8-sig', index=False)
client_df

In [ ]:
#데이터 정리(for 클러스터링)
df = pd.read_csv('final_client_df.csv')

en = LabelEncoder()
df['연령대'] = en.fit_transform(df['연령대'])

sc = MinMaxScaler()
df['연령대'] = pd.DataFrame(sc.fit_transform(df[['연령대']]), columns=['연령대'])

df['성별'].replace({'남성': 1, '여성': 0}, inplace=True)
df_encoded = pd.get_dummies(df[['거주지', '제휴사']])

df = pd.concat([df, df_encoded], axis=1)
df.index = df['고객번호']
df.drop(['고객번호', '거주지', '제휴사'], axis=1, inplace=True)

features = ['R', 'F', 'M1', 'M2', 'Lpay', '성별', '연령대']
df = df[features]

In [ ]:
# 클러스터링
##1. 원데이터
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

X_features = df.values

# KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
labels = kmeans.fit_predict(X_features)
df["cluster_label"] = labels

# 실루엣 스코어
silhouette_kmeans = silhouette_score(X_features, labels)
print(f"실루엣 스코어: {silhouette_kmeans:.3f}")

visualize_silhouette([3,4,5], X_features)
visualize_kmeans_plot_multi([3,4,5], X_features)

##2. StandardScaler
from sklearn.preprocessing import StandardScaler

# StandardScaler
X_features = df.values
X_features_scaled = StandardScaler().fit_transform(X_features)

# KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
labels = kmeans.fit_predict(X_features_scaled)
df["cluster_label"] = labels

# 실루엣 스코어
silhouette_kmeans = silhouette_score(X_features_scaled, labels)
print(f"실루엣 스코어: {silhouette_kmeans:.3f}")

visualize_silhouette([3,4,5], X_features_scaled)
visualize_kmeans_plot_multi([3,4,5], X_features_scaled)

##3. 로그변환+표준화X

# 로그 변환
df['R_log'] = np.log1p(df['R'])
df['F_log'] = np.log1p(df['F'])
df['M1_log'] = np.log1p(df['M1'])
df['M2_log'] = np.log1p(df['M2'])

# Features
features = list(df.columns)
features.remove('R')
features.remove('F')
features.remove('M1')
features.remove('M2')

# StandardScaler
X_features = df[features].values

# KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
labels = kmeans.fit_predict(X_features)
df["cluster_label"] = labels

# 실루엣 스코어
silhouette_kmeans = silhouette_score(X_features, labels)
print(f"실루엣 스코어: {silhouette_kmeans:.3f}")

visualize_silhouette([3,4,5], X_features)
visualize_kmeans_plot_multi([3,4,5], X_features)

##4. 로그변환+표준화O

# 로그 변환
df['R_log'] = np.log1p(df['R'])
df['F_log'] = np.log1p(df['F'])
df['M1_log'] = np.log1p(df['M1'])
df['M2_log'] = np.log1p(df['M2'])

# Features
features = list(df.columns)
features.remove('R')
features.remove('F')
features.remove('M1')
features.remove('M2')

# StandardScaler
X_features = df[features].values
X_features_scaled = StandardScaler().fit_transform(X_features)

# KMeans
kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(X_features_scaled)
df["cluster_label"] = labels

# 실루엣 스코어
silhouette_kmeans = silhouette_score(X_features_scaled, labels)
print(f"실루엣 스코어: {silhouette_kmeans:.3f}")

visualize_silhouette([3,4,5], X_features_scaled)
visualize_kmeans_plot_multi([3,4,5], X_features_scaled)

In [ ]:
#최종 클러스터링 결과 분석
print(df['cluster_label'].value_counts())
display(df.groupby('cluster_label').mean())
temp = df.groupby(['cluster_label']).mean()
cluster_mean = temp.transpose()
mean_table = cluster_mean.div(cluster_mean.max(axis=1), axis= 0)
df.to_csv('final_cluster.csv', encoding='utf-8-sig')
fig = plt.figure(figsize=(15, 15))
sns.heatmap(mean_table, annot=True, fmt ='.2f', linewidths = 0.1)
plt.show()

# 고객별 상품 추천

In [ ]:
##라이브러리 로드
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import os
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
%cd /Users/diekim/Desktop/ETC/LPOINT

In [ ]:
##함수 정의
def make_train (matrix, percentage = .2):
    '''
    -----------------------------------------------------
    설명
    유저-아이템 행렬 (matrix)에서 
    1. 0 이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
    2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
    -----------------------------------------------------
    반환
    training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    test_set:     원본 유저-아이템 행렬의 복사본
    user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
    '''
    test_set = matrix.copy()
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample (nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(user_inds))

def auc_score(test, predictions):
    '''
    fpr, tpr를 이용해서 AUC를 계산하는 함수
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr,tpr)

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    가려진 정보가 있는 유저마다 AUC 평균을 구하는 함수
    ----------------------------------------
    input
    1. training_set: make_train 함수에서 만들어진 훈련 데이터 (일정 비율로 아이템 구매량이 0으로 가려진 데이터)
    2. prediction: implicit MF에서 나온 유저/아이템 별로 나온 예측 평점 행렬
    3. altered_users: make_train 함수에서 아이템 구매량이 0으로 가려진 유저
    4. test_set: make_train함수에서 만든 테스트 데이터
    ----------------------------------------
    반환
    추천 시스템 유저의 평균 auc
    인기아이템 기반 유저 평균 auc
    '''
    # 리스트 초기화
    store_auc = []
    popularity_auc = []
    
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # 모든 유저의 아이템별 구매횟수 합
    item_vecs = predictions[1] # 아이템 latent 벡터
    
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1) # 유저의 훈련데이터
        zero_inds = np.where(training_row == 0) # 가려진 아이템 Index
        
        # 가려진 아이템에 대한 예측
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # 가려진 아이템에 대한 실제값
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # 가려진 아이템에 대한 popularity (구매횟수 합)
        pop = pop_items[zero_inds]
        
        # AUC 계산 
        store_auc.append(auc_score(actual, pred))
        popularity_auc.append(auc_score(actual,pop))
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  

# def get_items_purchased(customer_id, mf_train, customer_list, products_list, item_lookup):
#     '''
#     특정 유저가 구매한 목록을 보여주는 함수
#     ----------------------------------------
#     INPUT
#     1. customer_id: 고객 ID
#     2. mf_train: 훈련 데이터 평점
#     3. customers_list: 훈련 데이터에 쓰인 고객 목록
#     4. products_list: 훈련 데이터에 쓰인 아이템 목록
#     5. item_lookup: 유니크한 아이템 ID와 설명을 담은 테이블
#     '''
#     cust_ind = np.where (customer_list == customer_id)[0][0]
#     purchased_ind = mf_train[cust_ind,:].nonzero()[1]
#     prod_codes = products_list[purchased_ind]
    
#     return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

# def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
#     '''
#     유저의 추천 아이템 반환
#     -----------------------------------------------------
#     INPUT
#     1. customer_id - Input the customer's id number that you want to get recommendations for
#     2. mf_train: 훈련 데이터
#     3. user_vecs: 행렬 분해에 쓰인 유저 벡터
#     4. item_vecs: 행렬 분해에 쓰인 아이템 벡터
#     5. customer_list: 평점 행렬의 행에 해당하는 고객 ID
#     6. item_list: 평점 행렬의 열에 해당하는 아이템 ID
#     7. item_lookup: 아이템 ID와 설명을 담은 테이블
#     8. num_items: 추천할 아이템 개수
#     -----------------------------------------------------
#     반환    
#     구매한 적이 없는 아이템 중 예측 평점이 높은 최고 n개의 추천 아이템
#     '''
    
#     cust_ind = np.where(customer_list == customer_id)[0][0]
#     pref_vec = mf_train[cust_ind,:].toarray()                   # 훈련 데이터의 실제 평점
#     pref_vec = pref_vec.reshape(-1) + 1                         # 1을 더해서 환불한 것도 구매한 걸로 간주
#     pref_vec[pref_vec > 1] = 0                                  # 구매한 것들을 모두 0으로 
#     rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T)         # 추천 시스템에 기반한 예측 평점
    
#     # Min-Max Scaling
#     min_max = MinMaxScaler()
#     rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
#     recommend_vector = pref_vec*rec_vector_scaled  # 구매하지 않은 아이템에 대해서만 예측 평점이 남도록
    
#     product_idx = np.argsort(recommend_vector)[::-1][:num_items] # num_items만큼 내림차순으로 평점 정렬한 index
    
#     rec_list = []
    
#     for index in product_idx:
#         code = item_list[index] # 아이템 id
#         # id와 description 담기
#         rec_list.append([code, item_lookup['Description'].loc[item_lookup['pd_c'] == code].iloc[0]]) 
    
#     codes = [item[0] for item in rec_list]
#     descriptions = [item[1] for item in rec_list]
#     final_frame = pd.DataFrame({'pd_c': codes, 'Description': descriptions})
    
#     return final_frame[['pd_c', 'Description']]

def recommendation(usernum, cluster, n=10):
    userid = grouped[grouped['고객번호']==usernum]['userid'].values[0]
    recommendations = model.recommend(userid, csr_mat[userid], N=n)
    item_df = df[['상품코드', '대분류명', '중분류명', '소분류명']]
    res_df = pd.DataFrame()
    for i in range(n):
        print('userid: {}의 TOP{} 아이템 추천 테이블 생성 중...'.format(userid, i+1))
        rec_df = grouped[grouped['itemid']==recommendations[0][i]]['상품코드'].drop_duplicates()
        tmp_df = pd.merge(rec_df, item_df, on='상품코드', how='left').drop_duplicates()
        tmp_df['구매확률'] = recommendations[1][i]
        res_df = pd.concat([res_df, tmp_df], axis=0)
        res_df['고객번호'] = usernum
        res_df['클러스터'] = cluster
        res_df = res_df.reset_index(drop=True)
        #res_df.to_csv('{}_top10.csv'.format(userid), encoding='utf-8-sig')
    return res_df

In [ ]:
##데이터 정리
df = pd.read_csv('products.csv')

df.rename(columns={'대분류명': '중분류명',
                   '중분류명': '대분류명'},
          inplace=True)

grouped = df.groupby(['고객번호', '상품코드'])['구매수량'].sum().reset_index()

customers = list(np.sort(grouped['고객번호'].unique()))
products = list (grouped['상품코드'].unique())
quantity = list(grouped['구매수량'])

rows = grouped['고객번호'].astype('category').cat.codes
cols = grouped['상품코드'].astype('category').cat.codes

grouped['userid'] = rows
grouped['itemid'] = cols

print(len(customers)) 
print(len(products))  

csr_mat = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(customers),len(products)))
matrix_size = csr_mat.shape[0]* csr_mat.shape[1]
num_purchases = len(csr_mat.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
print(sparsity)

In [ ]:
##모델 적합
product_train, product_test, product_users_altered = make_train(csr_mat, 0.2)
model = implicit.als.AlternatingLeastSquares(factors=20,
                                             regularization=0.1,
                                             iterations=30)
alpha = 15
model.fit((product_train*alpha).astype('double'))

user_vecs = model.user_factors
item_vecs = model.item_factors
predictions = [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)]
print(calc_mean_auc(product_train, product_users_altered, predictions, product_test))

In [ ]:
##최종 추천
df1 = recommendation(usernum='M124357021', cluster=0, n=10) 
df2 = recommendation(usernum='M282063613', cluster=0, n=10) 
df3 = recommendation(usernum='M576689847', cluster=0, n=10) 
df4 = recommendation(usernum='M057015266', cluster=1, n=10) 
df5 = recommendation(usernum='M596502154', cluster=1, n=10) 
df6 = recommendation(usernum='M378415176', cluster=1, n=10) 
df7 = recommendation(usernum='M408936009', cluster=2, n=10) 
df8 = recommendation(usernum='M652598612', cluster=2, n=10) 
df9 = recommendation(usernum='M865603201', cluster=2, n=10) 
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)
df.to_csv('final_recsys.csv', encoding='utf-8-sig', index=False)

# 고객별 재구매 예측

## LSTM

In [ ]:
# pip install tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf
import datetime

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products_cltv.csv', index_col = 0)

In [ ]:
df['고객번호'].value_counts()

In [ ]:
df[df['고객번호']=='M057015266']['대분류명'].value_counts()

In [ ]:
id= 'M057015266'
product = '과자'

### 날짜만

#### 데이터프레임 생성

In [ ]:
data = df[(df['고객번호']==id) & (df['대분류명']==product)]
len(data)

In [ ]:
data['구매일자'] = data['구매일자'].astype('str')
data['구매일자'] = pd.to_datetime(data['구매일자'])

In [ ]:
# lpay 사용, 미사용 데이터프레임 나눠서 사용금액 나누기
lpay = data[data['Lpay']==1]
lpay_X = data[data['Lpay']==0]

lpay['lpay_구매금액'] = lpay['구매금액']
lpay['lpay_구매금액X'] = 0
lpay_X['lpay_구매금액'] = 0
lpay_X['lpay_구매금액X'] = lpay_X['구매금액']

data = pd.concat([lpay, lpay_X], axis = 0)
data = data.sort_index()

In [ ]:
data = data.groupby('구매일자').agg({'고객번호':'count',
                         '채널구분(1:오프, 2:온)':'mean',
                         'Lpay':'sum',
                         'lpay_구매금액':'sum',
                         '구매시간':'mean',
                         '구매수량':'sum',
                         '구매금액':'sum'})
data.columns = ['거래건수','온/오프_평균','Lpay거래건수','Lpay구매금액_총합','구매시간_평균','구매수량','구매금액_총합']
data = data[['거래건수','온/오프_평균','Lpay거래건수','Lpay구매금액_총합','구매시간_평균','구매금액_총합','구매수량']]
data.head()

In [ ]:
# 중간 날짜 데이터 채우기
data_index_list = data.index.tolist()
for i in range(len(data_index_list)-1) : 
    start_date = pd.to_datetime(str(data_index_list[i])[:10]) ## 시작 날짜
    end_date = pd.to_datetime(str(data_index_list[i+1])[:10]) ## 마지막 날짜
    dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성
    
    tt = pd.DataFrame(index = dates, columns = data.columns)
    tt = tt.fillna(0)
    tt = tt[1:-1]
    data = pd.concat([data, tt], axis = 0)
    data = data.sort_index()

In [ ]:
data

In [ ]:
DATE = data.index

#### LSTM

In [ ]:
x_1 = data['거래건수'].values
x_2 = data['온/오프_평균'].values
x_3 = data['Lpay거래건수'].values
x_4 = data['Lpay구매금액_총합'].values
x_5 = data['구매시간_평균'].values
x_6 = data['구매금액_총합'].values
y = data['구매수량'].values

In [ ]:
# Step 1 : convert to [rows, columns] structure
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_3.reshape((len(x_3), 1))
x_4 = x_3.reshape((len(x_4), 1))
x_5 = x_3.reshape((len(x_5), 1))
x_6 = x_3.reshape((len(x_6), 1))

y = y.reshape((len(y), 1))

In [ ]:
print ("x_1.shape" , x_1.shape) 
print ("x_2.shape" , x_2.shape) 
print ("x_3.shape" , x_3.shape) 
print ("y.shape" , y.shape)

In [ ]:
pip install varname

In [ ]:
def MinMaxScaler(x) : 
  x_MinMax = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
  return x_MinMax

In [ ]:
min_dict = {}
max_dict = {}
min_dict['x_1'] = x_1.min(axis = 0)[0]
max_dict['x_1'] = x_1.max(axis = 0)[0]
min_dict['x_2'] = x_2.min(axis = 0)[0]
max_dict['x_2'] = x_2.max(axis = 0)[0]
min_dict['x_3'] = x_3.min(axis = 0)[0]
max_dict['x_3'] = x_3.max(axis = 0)[0]
min_dict['x_4'] = x_4.min(axis = 0)[0]
max_dict['x_4'] = x_4.max(axis = 0)[0]
min_dict['x_5'] = x_5.min(axis = 0)[0]
max_dict['x_5'] = x_5.max(axis = 0)[0]
min_dict['x_6'] = x_6.min(axis = 0)[0]
max_dict['x_6'] = x_6.max(axis = 0)[0]
min_dict['y'] = y.min(axis = 0)[0]
max_dict['y'] = y.max(axis = 0)[0]

In [ ]:
min_dict

In [ ]:
# Step 2 : normalization 
# scaler = MinMaxScaler(feature_range=(0, 1))
x_1_scaled = MinMaxScaler_(x_1)
x_2_scaled = MinMaxScaler_(x_2)
x_3_scaled = MinMaxScaler_(x_3)
x_4_scaled = MinMaxScaler_(x_4)
x_5_scaled = MinMaxScaler_(x_5)
x_6_scaled = MinMaxScaler_(x_6)

y_scaled = MinMaxScaler_(y)

In [ ]:
# Step 3 : horizontally stack columns
dataset_stacked = hstack((x_1_scaled, x_2_scaled, x_3_scaled, x_4_scaled, x_5_scaled, x_6_scaled, y_scaled))
print ("dataset_stacked.shape" , dataset_stacked.shape)

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
 X, y = list(), list()
 for i in range(len(sequences)):
  # find the end of this pattern
  end_ix = i + n_steps_in
  out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
  if out_end_ix > len(sequences):
   break
  # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)
# choose a number of time steps #change this accordingly
n_steps_in, n_steps_out = 1 , 1
# covert into input/output
X, y = split_sequences(dataset_stacked, n_steps_in, n_steps_out)
print ("X.shape" , X.shape) 
print ("y.shape" , y.shape)

In [ ]:
split_point = int(len(data) * 0.9)
train_X , train_y = X[:split_point, :] , y[:split_point, :]
test_X , test_y = X[split_point:, :] , y[split_point:, :]

In [ ]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 6)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])

In [ ]:
# Fit network
history = model.fit(train_X , train_y , epochs=100 , steps_per_epoch=25 , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False)

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model.evaluate(train_X, train_y)

In [ ]:
pred = model.predict(test_X)

In [ ]:
xx = pd.DataFrame(train_y, index = data.index[:split_point])
yy = pd.DataFrame(test_y, index = data.index[split_point:])
true = pd.concat([xx, yy], axis = 0)
pred = pd.DataFrame(pred, index = data.index[split_point:])
pred[0] = pred[0].apply(lambda x : x if x>=0 else 0)

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(true, label = 'TRUE VALUE')
plt.plot(pred, label = 'PREDICT VALUE')
plt.xlim(datetime.datetime(2021,11,1),datetime.datetime(2022,1,15))
plt.ylim(0,0.6)
plt.legend()

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(yy, pred)

In [ ]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(yy, pred)

In [ ]:
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(yy, pred) 
np.sqrt(MSE)

In [ ]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(yy, pred)

In [ ]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(yy, pred)

In [ ]:
def MAE(y_test, y_pred): 
    return np.mean((y_test - y_pred) / y_test) * 100
    
MAE(yy, pred)

In [ ]:
def MinMax_to_Raw(x_MinMax, min_value, max_value) : 
  x = map(int, x_MinMax * (max_value - min_value) +  min_value)
  return list(x)

In [ ]:
y_raw = pd.DataFrame(MinMax_to_Raw(y_scaled, min_dict['y'], max_dict['y']), index = data.index)
y_pred = pd.DataFrame(MinMax_to_Raw(pred, min_dict['y'], max_dict['y']), index = data.index[split_point:])

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(y_raw, label = 'real value')
plt.plot(y_pred, label = 'predicted')
plt.legend()

### 날짜 + 시간

#### 데이터프레임 생성

In [ ]:
data = df[(df['고객번호']==id) & (df['대분류명']==product)]
len(data)

In [ ]:
# 시간 변환 함수1: 1 -> 01:00:00 
def convert_time1(x):
    if x<10:
      return '0'+str(x)+':00:00'
    else:
      return str(x)+':00:00'

# 시간 변환
data['구매시간'] = data['구매시간'].apply(lambda x:convert_time1(x))
data = data.reset_index(drop = True)

In [ ]:
# time 변수 생성

for i in range(len(data)):
  data['구매일자'][i] = str(data['구매일자'][i]) + ' ' + data['구매시간'][i]
data

In [ ]:
data['구매일자'] = data['구매일자'].astype('str')
data['구매일자'] = pd.to_datetime(data['구매일자'])

In [ ]:
# lpay 사용, 미사용 데이터프레임 나눠서 사용금액 나누기
lpay = data[data['Lpay']==1]
lpay_X = data[data['Lpay']==0]

lpay['lpay_구매금액'] = lpay['구매금액']
lpay['lpay_구매금액X'] = 0
lpay_X['lpay_구매금액'] = 0
lpay_X['lpay_구매금액X'] = lpay_X['구매금액']

data = pd.concat([lpay, lpay_X], axis = 0)
data = data.sort_index()

In [ ]:
data = data.groupby('구매일자').agg({'고객번호':'count',
                         '채널구분(1:오프, 2:온)':'mean',
                         'Lpay':'sum',
                         'lpay_구매금액':'sum',
                         '구매수량':'sum',
                         '구매금액':'sum'})
data.columns = ['거래건수','온/오프_평균','Lpay거래건수','Lpay구매금액_총합','구매수량','구매금액_총합']
data = data[['거래건수','온/오프_평균','Lpay거래건수','Lpay구매금액_총합','구매금액_총합','구매수량']]
data

In [ ]:
# 중간 날짜 데이터 채우기
data_index_list = data.index.tolist()
for i in range(len(data_index_list)-1) : 
  start_date = data.index[i] ## 시작 날짜
  end_date = data.index[i+1] ## 마지막 날짜
  dates = pd.date_range(start_date,end_date,freq='H') ## 시간 단위로 생성
  
  tt = pd.DataFrame(index = dates, columns = data.columns)
  tt = tt.fillna(0)
  tt = tt[1:-1]
  data = pd.concat([data, tt], axis = 0)
data = data.sort_index()

In [ ]:
data

In [ ]:
data[['구매수량']]

In [ ]:
DATE = data.index

#### LSTM

In [ ]:
x_1 = data['거래건수'].values
x_2 = data['온/오프_평균'].values
x_3 = data['Lpay거래건수'].values
x_4 = data['Lpay구매금액_총합'].values
x_5 = data['구매금액_총합'].values
y = data['구매수량'].values

In [ ]:
# Step 1 : convert to [rows, columns] structure
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_3.reshape((len(x_3), 1))
x_4 = x_3.reshape((len(x_4), 1))
x_5 = x_3.reshape((len(x_5), 1))

y = y.reshape((len(y), 1))

In [ ]:
print ("x_1.shape" , x_1.shape) 
print ("x_2.shape" , x_2.shape) 
print ("x_3.shape" , x_3.shape) 
print ("y.shape" , y.shape)

In [ ]:
# pip install varname

In [ ]:
def MinMaxScaler(x) : 
  x_MinMax = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
  return x_MinMax

In [ ]:
min_dict = {}
max_dict = {}
min_dict['x_1'] = x_1.min(axis = 0)[0]
max_dict['x_1'] = x_1.max(axis = 0)[0]
min_dict['x_2'] = x_2.min(axis = 0)[0]
max_dict['x_2'] = x_2.max(axis = 0)[0]
min_dict['x_3'] = x_3.min(axis = 0)[0]
max_dict['x_3'] = x_3.max(axis = 0)[0]
min_dict['x_4'] = x_4.min(axis = 0)[0]
max_dict['x_4'] = x_4.max(axis = 0)[0]
min_dict['x_5'] = x_5.min(axis = 0)[0]
max_dict['x_5'] = x_5.max(axis = 0)[0]
min_dict['y'] = y.min(axis = 0)[0]
max_dict['y'] = y.max(axis = 0)[0]

In [ ]:
max_dict

In [ ]:
# Step 2 : normalization 
# scaler = MinMaxScaler(feature_range=(0, 1))
x_1_scaled = MinMaxScaler_(x_1)
x_2_scaled = MinMaxScaler_(x_2)
x_3_scaled = MinMaxScaler_(x_3)
x_4_scaled = MinMaxScaler_(x_4)
x_5_scaled = MinMaxScaler_(x_5)

y_scaled = MinMaxScaler_(y)

In [ ]:
# Step 3 : horizontally stack columns
dataset_stacked = hstack((x_1_scaled, x_2_scaled, x_3_scaled, x_4_scaled, x_5_scaled, y_scaled))
print ("dataset_stacked.shape" , dataset_stacked.shape)

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
 X, y = list(), list()
 for i in range(len(sequences)):
  # find the end of this pattern
  end_ix = i + n_steps_in
  out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
  if out_end_ix > len(sequences):
   break
  # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)
# choose a number of time steps #change this accordingly
n_steps_in, n_steps_out = 1 , 1
# covert into input/output
X, y = split_sequences(dataset_stacked, n_steps_in, n_steps_out)
print ("X.shape" , X.shape) 
print ("y.shape" , y.shape)

In [ ]:
split_point = int(len(data) * 0.9)
train_X , train_y = X[:split_point, :] , y[:split_point, :]
test_X , test_y = X[split_point:, :] , y[split_point:, :]

In [ ]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 5)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])

In [ ]:
# Fit network
history = model.fit(train_X , train_y , epochs=100 , steps_per_epoch=25 , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False)

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model.evaluate(train_X, train_y)

In [ ]:
pred = model.predict(test_X)

In [ ]:
len(data.index[split_point:])

In [ ]:
split_point

In [ ]:
len(train_y)

In [ ]:
len(true)

In [ ]:
xx = pd.DataFrame(train_y, index = data.index[:split_point])
yy = pd.DataFrame(test_y, index = data.index[split_point:])
true = pd.concat([xx, yy], axis = 0)
pred = pd.DataFrame(pred, index = data.index[split_point:])
pred[0] = pred[0].apply(lambda x : x if x>=0 else 0)

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(true, label = 'TRUE VALUE')
plt.plot(pred, label = 'PREDICT VALUE')
plt.xlim(datetime.datetime(2021,11,1),datetime.datetime(2022,1,1))
plt.ylim(0,0.6)
plt.legend()

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(yy, pred)

In [ ]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(yy, pred)

In [ ]:
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(yy, pred) 
np.sqrt(MSE)

In [ ]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(yy, pred)

In [ ]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(yy, pred)

In [ ]:
def MAE(y_test, y_pred): 
    return np.mean((y_test - y_pred) / y_test) * 100
    
MAE(yy, pred)

In [ ]:
def MinMax_to_Raw(x_MinMax, min_value, max_value) : 
  x = map(int, x_MinMax * (max_value - min_value) +  min_value)
  return list(x)

In [ ]:
y_raw = pd.DataFrame(MinMax_to_Raw(y_scaled, min_dict['y'], max_dict['y']), index = data.index)
y_pred = pd.DataFrame(MinMax_to_Raw(np.array(pred), min_dict['y'], max_dict['y']), index = data.index[split_point:])

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(y_raw, label = 'real value')
plt.plot(y_pred, label = 'predicted')
plt.legend()

## Prophet

In [ ]:
!pip install pystan

In [ ]:
pip install --upgrade plotly

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
from prophet import Prophet

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='Malgun Gothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="Malgun Gothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

### 1. 구매일자만 (구매시간 X)

In [ ]:
import pandas as pd

product = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products.csv')
product.set_index('고객번호',inplace=True)
product.rename(columns={'대분류명':'중분류명','중분류명':'대분류명'},inplace=True)
product

In [ ]:
# 작성날짜 datetime으로 변환
product['구매일자'] = product['구매일자'].astype('str')
product['구매일자'] = pd.to_datetime(product['구매일자'])

In [ ]:
product.loc['M057015266']['대분류명'].unique()

In [ ]:
customer1 = product.loc['M057015266'][product.loc['M057015266']['대분류명']=='과자'] ## 소분류명 구매수량 예측이 의미있을까,,? 너무 적은 데이터인데... -> 대분류명으로 해보기!
customer1

In [ ]:
customer1_time = pd.DataFrame(customer1.groupby('구매일자')['구매수량'].sum().reset_index())
customer1_time.rename(columns={'구매일자':'ds','구매수량':'y'}, inplace=True)
customer1_time

In [ ]:
customer1_time.set_index('ds').plot()

In [ ]:
m = Prophet()
m.fit(customer1_time)

In [ ]:
# make_future_dataframe() 함수를 이용해 예측값을 넣을 데이터 프레임을 생성
# periods 값은 향후 몇일 (또는 주,월 등 단위 주기) 을 예측할 것인지를 의미 -> periods = 30 : 30일의 실적을 예측
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig1 = m.plot(forecast) # 시각화

In [ ]:
# 피팅된 모델의 컴포넌트들을 시각화

fig2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot

m = Prophet(changepoint_prior_scale=0.05)
m.fit(customer1_time)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
m = Prophet(changepoint_prior_scale=0.3)
m.fit(customer1_time)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

### 2. 구매일자 + 구매시간

In [ ]:
import pandas as pd

product = pd.read_csv('/content/drive/MyDrive/Project/lotte/data/products.csv')
product.set_index('고객번호',inplace=True)
product.rename(columns={'대분류명':'중분류명','중분류명':'대분류명'},inplace=True)
product

In [ ]:
# 작성날짜 datetime으로 변환
product['구매일자'] = product['구매일자'].astype('str')
product['구매일자'] = pd.to_datetime(product['구매일자'])

In [ ]:
product.loc['M057015266']['대분류명'].unique()

In [ ]:
customer1 = product.loc['M057015266'][product.loc['M057015266']['대분류명']=='과자'] ## 소분류명 구매수량 예측이 의미있을까,,? 너무 적은 데이터인데... -> 대분류명으로 해보기!
customer1

In [ ]:
# 시간 변환 함수1: 1 -> 01:00:00 
def convert_time1(x):
    if x<10:
      return '0'+str(x)+':00:00'
    else:
      return str(x)+':00:00'

# 시간 변환
customer1['구매시간'] = customer1['구매시간'].apply(lambda x:convert_time1(x))
customer1

In [ ]:
# time 변수 생성

for i in range(len(customer1)):
  customer1['구매일자'][i] = str(customer1['구매일자'][i]) + ' ' + customer1['구매시간'][i]
customer1

In [ ]:
customer1['구매일자'] = pd.to_datetime(customer1['구매일자'])
customer1

In [ ]:
customer1_time = pd.DataFrame(customer1.groupby('구매일자')['구매수량'].sum().reset_index())
customer1_time.rename(columns={'구매일자':'ds','구매수량':'y'}, inplace=True)
customer1_time

In [ ]:
customer1_time.set_index('ds').plot()

In [ ]:
m = Prophet()
m.fit(customer1_time)

In [ ]:
# make_future_dataframe() 함수를 이용해 예측값을 넣을 데이터 프레임을 생성
# periods 값은 향후 몇일 (또는 주,월 등 단위 주기) 을 예측할 것인지를 의미 -> periods = 30 : 30일의 실적을 예측
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig1 = m.plot(forecast) # 시각화

In [ ]:
# 피팅된 모델의 컴포넌트들을 시각화

fig2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot

m = Prophet(changepoint_prior_scale=0.05)
m.fit(customer1_time)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
m = Prophet(changepoint_prior_scale=0.3)
m.fit(customer1_time)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
m = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    # seasonality
    weekly_seasonality=20,
    daily_seasonality=False
)

m.fit(customer1_time)
fig = m.plot_components(forecast)

In [ ]:
m = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    # seasonality
    weekly_seasonality=10,
    daily_seasonality=10
)

m.fit(customer1_time)
fig = m.plot_components(forecast)

In [ ]:
ms = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    # seasonality
    weekly_seasonality=10,
    daily_seasonality=10
)

ms.add_seasonality(name='monthly', period=30.5, fourier_order=5)

ms.fit(customer1_time)
fig = ms.plot_components(forecast)

### 3. train,test 나눠서 예측 비교

In [ ]:
customer1_time

In [ ]:
train = customer1_time[:40]
test = customer1_time[40:]

In [ ]:
def utils_evaluate_forecast(dtf, title, plot=True, figsize=(20,13)):
    try:
        ## residuals
        dtf["residuals"] = dtf["ts"] - dtf["model"]
        dtf["error"] = dtf["ts"] - dtf["forecast"]
        dtf["error_pct"] = dtf["error"] / dtf["ts"]
        
        ## kpi
        residuals_mean = dtf["residuals"].mean()
        residuals_std = dtf["residuals"].std()
        error_mean = dtf["error"].mean()
        error_std = dtf["error"].std()
        mae = dtf["error"].apply(lambda x: np.abs(x)).mean()
        mape = dtf["error_pct"].apply(lambda x: np.abs(x)).mean()  
        mse = dtf["error"].apply(lambda x: x**2).mean()
        rmse = np.sqrt(mse)  #root mean squared error
        
        ## intervals
        dtf["conf_int_low"] = dtf["forecast"] - 1.96*residuals_std
        dtf["conf_int_up"] = dtf["forecast"] + 1.96*residuals_std
        dtf["pred_int_low"] = dtf["forecast"] - 1.96*error_std
        dtf["pred_int_up"] = dtf["forecast"] + 1.96*error_std
        
        ## plot
        if plot==True:
            fig = plt.figure(figsize=figsize)
            fig.suptitle(title, fontsize=20)   
            ax1 = fig.add_subplot(2,2, 1)
            ax2 = fig.add_subplot(2,2, 2, sharey=ax1)
            ax3 = fig.add_subplot(2,2, 3)
            ax4 = fig.add_subplot(2,2, 4)
            ### training
            dtf[pd.notnull(dtf["model"])][["ts","model"]].plot(color=["black","green"], title="Model", grid=True, ax=ax1)      
            ax1.set(xlabel=None)
            ### test
            dtf[pd.isnull(dtf["model"])][["ts","forecast"]].plot(color=["black","red"], title="Forecast", grid=True, ax=ax2)
            ax2.fill_between(x=dtf.index, y1=dtf['pred_int_low'], y2=dtf['pred_int_up'], color='b', alpha=0.2)
            ax2.fill_between(x=dtf.index, y1=dtf['conf_int_low'], y2=dtf['conf_int_up'], color='b', alpha=0.3)     
            ax2.set(xlabel=None)
            ### residuals
            dtf[["residuals","error"]].plot(ax=ax3, color=["green","red"], title="Residuals", grid=True)
            ax3.set(xlabel=None)
            ### residuals distribution
            dtf[["residuals","error"]].plot(ax=ax4, color=["green","red"], kind='kde', title="Residuals Distribution", grid=True)
            ax4.set(ylabel=None)
            plt.show()
            print("Training --> Residuals mean:", np.round(residuals_mean), " | std:", np.round(residuals_std))
            print("Test --> Error mean:", np.round(error_mean), " | std:", np.round(error_std),
                  " | mae:",np.round(mae), " | mape:",np.round(mape*100), "%  | mse:",np.round(mse), " | rmse:",np.round(rmse))
        
        return dtf[["ts","model","residuals","conf_int_low","conf_int_up", 
                    "forecast","error","pred_int_low","pred_int_up"]]
    
    except Exception as e:
        print("--- got error ---")
        print(e)

In [ ]:
def fit_prophet(dtf_train, dtf_test, lst_exog=None, model=None, 
                freq="D", figsize=(15,10)):
    ## train
    model.fit(dtf_train) ####
    
    ## test
    dtf_prophet = model.make_future_dataframe(periods=len(dtf_test), 
                  freq=freq, include_history=True)
    dtf_prophet = model.predict(dtf_prophet)
    dtf_train = dtf_train.merge(dtf_prophet[["ds","yhat"]], 
                how="left").rename(columns={'yhat':'model', 
                'y':'ts'}).set_index("ds")
    #dtf_test = dtf_test.merge(dtf_prophet[["ds","yhat"]], 
    #            how="left").rename(columns={'yhat':'forecast',  
    #            'y':'ts'}).set_index("ds")
    dtf_test = pd.DataFrame(dtf_prophet[["ds","yhat"]]['yhat'][40:])
    dtf_test = pd.concat([test,dtf_test],axis=1)
    dtf_test.set_index('ds',inplace=True)
    dtf_test.rename(columns={'yhat':'forecast','y':'ts'},inplace=True)
    
    ## evaluate
    dtf = dtf_train.append(dtf_test)
    #dtf = pd.concat([dtf_train,dtf_test],axis=0)
    dtf = utils_evaluate_forecast(dtf, figsize=figsize, 
                                  title="Prophet")

    return dtf, model

In [ ]:
model = Prophet(growth="linear", changepoints=None, 
                n_changepoints=25,
                seasonality_mode="multiplicative",
                yearly_seasonality=True, 
                weekly_seasonality="auto", 
                daily_seasonality=False,
                holidays=None)

In [ ]:
dtf, model = fit_prophet(train, test, model=model, freq="D")

In [ ]:
model1 = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    # seasonality
    weekly_seasonality=10,
    daily_seasonality=10
)

dtf1, model1 = fit_prophet(train, test, model=model1, freq="D")

In [ ]:
model2 = Prophet(
    # trend
    changepoint_prior_scale=0.3,
    # seasonality
    weekly_seasonality=10,
    daily_seasonality=10
)

model2.add_seasonality(name='monthly', period=30.5, fourier_order=5)

dtf2, model2 = fit_prophet(train, test, model=model2, freq="D")

## ARIMA

In [ ]:
import pandas as pd
pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products.csv')

In [ ]:
!pip install pmdarima

In [ ]:
!pip install scalecast

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='Malgun Gothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="Malgun Gothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

### CLTV 값 있는 고객만 골라내기

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products.csv')
df.rename(columns={'대분류명':'중분류명','중분류명':'대분류명'},inplace=True)

In [ ]:
cltv = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/final_cltv.csv')
cltv.head()

In [ ]:
cltv_client = cltv['고객번호'].unique().tolist()
df['고객번호'] = df['고객번호'].apply(lambda x : x if x in cltv_client else 0)

In [ ]:
df = df[df['고객번호']!=0]
df.head()

In [ ]:
df.to_csv('/content/gdrive/MyDrive/Project/lotte/data/products_cltv.csv')

### 대상 고객, 상품 선택

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products_cltv.csv', index_col = 0)

M057015266 고객의 건강용품 구매수량 예측

In [ ]:
df['고객번호'].value_counts()

In [ ]:
df[df['고객번호']=='M057015266']['대분류명'].value_counts()

In [ ]:
id= 'M057015266'
product = '과자'

In [ ]:
data = df[(df['고객번호']==id) & (df['대분류명']==product)]
len(data)

In [ ]:
data['구매일자'] = data['구매일자'].astype('str')
data['구매일자'] = pd.to_datetime(data['구매일자'])

In [ ]:
data = data.groupby('구매일자')[['구매수량']].sum()

In [ ]:
# 중간 날짜 데이터 채우기
data_index_list = data.index.tolist()
for i in range(len(data_index_list)-1) : 
    start_date = pd.to_datetime(str(data_index_list[i])[:10]) ## 시작 날짜
    end_date = pd.to_datetime(str(data_index_list[i+1])[:10]) ## 마지막 날짜
    dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성
    
    tt = pd.DataFrame(index = dates, columns = data.columns)
    tt = tt.fillna(0)
    tt = tt[1:-1]
    data = pd.concat([data, tt], axis = 0)
    data = data.sort_index()

In [ ]:
data

In [ ]:
DATE = data.index

### ARIMA 시작

In [ ]:
import pandas as pd
import numpy as np
from scalecast.Forecaster import Forecaster
from scalecast import GridGenerator
from pmdarima import auto_arima
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(14,7)})

##### 원본

In [ ]:
# f = Forecaster(y=data['구매수량'],current_dates=data.index)

In [ ]:
# f.generate_future_dates(12) # 12-month forecast horizon
# f.set_test_length(.2) # 20% test set
# f.set_estimator('arima') # set arima
# f.manual_forecast(call_me='arima1') # forecast with arima

# f.plot_test_set(ci=True) # view test results
# plt.title('ARIMA Test-Set Performance',size=14)
# plt.show()

In [ ]:
# # EDA
# f.plot_acf()
# plt.show()
# f.plot_pacf()
# plt.show()
# f.seasonal_decompose().plot()
# plt.show()
# stat, pval, _, _, _, _ = f.adf_test(full_res=True)
# print(stat)
# print(pval)

# # Forecast
# f.manual_forecast(order=(1,1,1),seasonal_order=(2,1,1,12),call_me='arima2')

# # View test results
# f.plot_test_set(ci=True,models='arima2')
# plt.title('ARIMA Test-Set Performance',size=14)
# plt.show()

# # View forecast results
# f.plot(ci=True,models='arima2')
# plt.title('ARIMA Forecast Performance',size=14)
# plt.show()

# # See summary stats
# f.regr.summary()

##### MinMaxScaler_사용

In [ ]:
def MinMaxScaler_(x) : 
  x_MinMax = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
  return x_MinMax

In [ ]:
## 정상성을 해결하기 위한 1차 차분
## minmaxscale
import numpy
x = data['구매수량'].values
x_scaled = MinMaxScaler_(x)

In [ ]:
min_value = x.min(axis = 0)
max_value = x.max(axis = 0)

In [ ]:
f = Forecaster(y=x_scaled, current_dates=data.index)

In [ ]:
f.generate_future_dates(12) # 12-month forecast horizon
f.set_test_length(.2) # 20% test set
f.set_estimator('arima') # set arima
f.manual_forecast(call_me='arima1') # forecast with arima

f.plot_test_set(ci=True) # view test results
plt.title('ARIMA Test-Set Performance',size=14)
plt.show()

* ACF : 자기상관함수(Autocorrelation Fucntion)
  * y(t)와 y(t+k) 사이에 correlation을 측정
  * 이 값이 가장 작은 모델 선정

In [ ]:
# EDA
f.plot_acf()
plt.show()
f.plot_pacf()
plt.show()
f.seasonal_decompose().plot()
plt.show()
stat, pval, _, _, _, _ = f.adf_test(full_res=True)
print(stat)
print(pval)

# Forecast
f.manual_forecast(order=(1,1,1),seasonal_order=(2,1,1,12),call_me='arima2')

# View test results
f.plot_test_set(ci=True,models='arima2')
plt.title('ARIMA Test-Set Performance',size=14)
plt.show()

# View forecast results
f.plot(ci=True,models='arima2')
plt.title('ARIMA Forecast Performance',size=14)
plt.show()

# See summary stats
f.regr.summary()

#### 공통

In [ ]:
from pmdarima import auto_arima
import warnings

warnings.filterwarnings('ignore')

train = data.iloc[:int(.8*(data.shape[0])),-1]
auto_model = auto_arima(
  train,
  start_P=1,
  start_q=1,
  max_p=6,
  max_q=6,m=12,
  seasonal=True,
  max_P=2, 
  max_D=2,
  max_Q=2,
  max_d=2,
  trace=True,
  error_action='ignore',
  suppress_warnings=True,
  stepwise=True,
  information_criterion="aic",
  alpha=0.05,
  scoring='mse'
)

best_params = auto_model.get_params()
order = best_params['order']
seasonal_order = best_params['seasonal_order']
trend = best_params['trend']

f.manual_forecast(order=order,seasonal_order=seasonal_order,trend=trend,call_me='arima3')

f.plot_test_set(ci=True,models='arima3')
plt.title('ARIMA Test-Set Performance',size=14)
plt.show()

f.plot(ci=True,models='arima3')
plt.title('ARIMA Forecast Performance',size=14)
plt.show()

In [ ]:
f.regr.summary()

In [ ]:
f.set_validation_length(12)
grid = {
    'order':[(1,1,1),(1,1,0),(0,1,1),(0,0,1),(6,0,4), (6,1,1), (0,0,0)],
    'seasonal_order':[(2,1,1,12),(1,1,1,12),(2,1,0,12),(0,1,0,12)]
}

f.ingest_grid(grid)
f.tune()
f.auto_forecast(call_me='arima3')

f.plot_test_set(ci=True,models='arima3')
plt.title('ARIMA Test-Set Performance',size=14)
plt.show()

f.plot(ci=True,models='arima3')
plt.title('ARIMA Forecast Performance',size=14)
plt.show()

f.regr.summary()

In [ ]:
pd.options.display.max_colwidth = 100
results = f.export(to_excel=False,
                   excel_name='/content/gdrive/MyDrive/Project/lotte/data/{}_{}_performance.xlsx'.format(id,product),
                   determine_best_by='TestSetMAPE')

In [ ]:
summaries = results['model_summaries']
summaries.to_csv('/content/gdrive/MyDrive/Project/lotte/data/{}_{}_performance.csv'.format(id,product))
summaries

In [ ]:
summaries.iloc[:,10:]

In [ ]:
f.export(cis = True)

In [ ]:
f.export()['test_set_predictions']

In [ ]:
dates = f.export()['test_set_predictions']['DATE']
real = f.export()['test_set_predictions']['actual']
pred = f.export()['test_set_predictions']['arima3']
pred_upper = f.export(cis = True)['test_set_predictions']['arima3_upperci']
pred_lower = f.export(cis = True)['test_set_predictions']['arima3_lowerci']
future_dates =  f.export()['all_fcsts']['DATE']
future = f.export()['all_fcsts']['arima3']
future_upper = f.export(cis = True)['all_fcsts']['arima3_upperci']
future_lower = f.export(cis = True)['all_fcsts']['arima3_lowerci']

In [ ]:
def MinMax_to_Raw(x_MinMax, min_value, max_value) : 
  x = map(int, x_MinMax * (max_value - min_value) +  min_value)
  return list(x)

In [ ]:
raw_data = pd.DataFrame(MinMax_to_Raw(real, min_value, max_value), index = dates)
pred_data = pd.DataFrame(MinMax_to_Raw(pred, min_value, max_value), index = dates)
pred_upper_data = pd.DataFrame(MinMax_to_Raw(pred_upper, min_value, max_value), index = dates)
pred_lower_data = pd.DataFrame(MinMax_to_Raw(pred_lower, min_value, max_value), index = dates)
future_data = pd.DataFrame(MinMax_to_Raw(future, min_value, max_value), index = future_dates)
future_upper_data = pd.DataFrame(MinMax_to_Raw(future_upper, min_value, max_value), index = future_dates)
future_lower_data = pd.DataFrame(MinMax_to_Raw(future_lower, min_value, max_value), index = future_dates)

In [ ]:
import datetime
dataset = pd.DataFrame(index = pd.date_range(data.index[0],future_dates[11],freq='D'), columns = ['raw','predicted','predicted_upperci','predicted_lowerci'])
dataset.loc[data.index, 'raw'] = data['구매수량']
dataset.loc[pred_data.index, 'predicted'] = pred_data[0].tolist()
dataset.loc[future_data.index, 'predicted'] = future_data[0].tolist()
dataset.loc[pred_upper_data.index, 'predicted_upperci'] = pred_upper_data[0].tolist()
dataset.loc[future_upper_data.index, 'predicted_upperci'] = future_upper_data[0].tolist()
dataset.loc[pred_lower_data.index, 'predicted_lowerci'] = pred_lower_data[0].tolist()
dataset.loc[future_lower_data.index, 'predicted_lowerci'] = future_lower_data[0].tolist()
dataset['predicted_upperci'] = dataset['predicted_upperci'].apply(lambda x : 0 if x<0 else x)
dataset['predicted_lowerci'] = dataset['predicted_lowerci'].apply(lambda x : 0 if x<0 else x)
dataset

In [ ]:
plt.figure(figsize = (25, 8))
plt.rc('xtick', labelsize=15)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=20)  # 범례 폰트 크기
plt.plot(dataset['raw'], label = 'raw')
plt.plot(dataset['predicted'], label = 'predicted')
plt.fill_between(dataset[dataset.index>=dates.iloc[0]].index, dataset.loc[dataset.index>=dates.iloc[0],'predicted_lowerci'].tolist(), dataset.loc[dataset.index>=dates.iloc[0],'predicted_upperci'].tolist(), color = 'orange', alpha = 0.3)
plt.legend()

plt.savefig('/content/gdrive/MyDrive/Project/lotte/data/{} 고객의 {} 구매수량 예측'.format(id,product))

In [ ]:
plt.figure(figsize = (20, 8))
plt.rc('xtick', labelsize=15)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=20)  # 범례 폰트 크기
plt.plot(dataset.loc[dataset.index>dates.iloc[0],'raw'], label = 'raw')
plt.plot(dataset['predicted'], label = 'predicted')
plt.fill_between(dataset[dataset.index>=dates.iloc[0]].index, dataset.loc[dataset.index>=dates.iloc[0],'predicted_lowerci'].tolist(), dataset.loc[dataset.index>=dates.iloc[0],'predicted_upperci'].tolist(), color = 'orange', alpha = 0.3)
plt.legend()

plt.savefig('/content/gdrive/MyDrive/Project/lotte/data/{} 고객의 {} 구매수량 예측(예측범위만)'.format(id,product))

In [ ]:
dataset.to_csv('/content/gdrive/MyDrive/Project/lotte/data/{}_{}_raw_predict_values.csv'.format(id,product))

### ARIMA 자동화

In [ ]:
import pandas as pd
import numpy as np
import datetime
from scalecast.Forecaster import Forecaster
from scalecast import GridGenerator
from pmdarima import auto_arima
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(14,7)})
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Project/lotte/data/products_cltv.csv', index_col = 0)

In [ ]:
def MinMaxScaler_(x) : 
  x_MinMax = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
  return x_MinMax

def MinMax_to_Raw(x_MinMax, min_value, max_value) : 
  x = map(int, x_MinMax * (max_value - min_value) +  min_value)
  return list(x)

In [ ]:
# 고객리스트
clients = ['M057015266', 'M124357021', 'M282063613','M576689847', 'M596502154', 'M378415176','M408936009', 'M652598612', 'M865603201']

In [ ]:
for i in range(len(clients)) : 
  id= clients[i]
  product = df[df['고객번호']==clients[i]]['대분류명'].value_counts().index[0]
  print(id,':',product)
  data = df[(df['고객번호']==id) & (df['대분류명']==product)]
  data['구매일자'] = data['구매일자'].astype('str')
  data['구매일자'] = pd.to_datetime(data['구매일자'])
  data = data.groupby('구매일자')[['구매수량']].sum()

  # 중간 날짜 데이터 채우기
  data_index_list = data.index.tolist()
  for i in range(len(data_index_list)-1) : 
      start_date = pd.to_datetime(str(data_index_list[i])[:10]) ## 시작 날짜
      end_date = pd.to_datetime(str(data_index_list[i+1])[:10]) ## 마지막 날짜
      dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성
      
      tt = pd.DataFrame(index = dates, columns = data.columns)
      tt = tt.fillna(0)
      tt = tt[1:-1]
      data = pd.concat([data, tt], axis = 0)
      data = data.sort_index()

  x = data['구매수량'].values
  x_scaled = MinMaxScaler_(x)
  min_value = x.min(axis = 0)
  max_value = x.max(axis = 0)

  f = Forecaster(y=x_scaled, current_dates=data.index)

  f.generate_future_dates(12) # 12-month forecast horizon
  f.set_test_length(.2) # 20% test set
  f.set_estimator('arima') # set arima
  f.manual_forecast(call_me='arima1') # forecast with arima

  f.manual_forecast(order=(1,1,1),seasonal_order=(2,1,1,12),call_me='arima2')

  train = data.iloc[:int(.8*(data.shape[0])),-1]
  auto_model = auto_arima(
    train,
    start_P=1,
    start_q=1,
    max_p=6,
    max_q=6,m=12,
    seasonal=True,
    max_P=2, 
    max_D=2,
    max_Q=2,
    max_d=2,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True,
    information_criterion="aic",
    alpha=0.05,
    scoring='mse'
  )

  best_params = auto_model.get_params()
  order = best_params['order']
  seasonal_order = best_params['seasonal_order']
  trend = best_params['trend']

  f.manual_forecast(order=order,seasonal_order=seasonal_order,trend=trend,call_me='arima3')
  f.set_validation_length(12)
  grid = {
      'order':[(1,1,1),(1,1,0),(0,1,1),(0,0,1),(6,0,4), (6,1,1), (0,0,0)],
      'seasonal_order':[(2,1,1,12),(1,1,1,12),(2,1,0,12),(0,1,0,12)]
  }
  f.ingest_grid(grid)
  f.tune()
  f.auto_forecast(call_me='arima3')

  pd.options.display.max_colwidth = 100
  results = f.export(to_excel=False,
                    excel_name='/content/gdrive/MyDrive/Project/lotte/data/{}_{}_performance.xlsx'.format(id,product),
                    determine_best_by='TestSetMAPE')
  summaries = results['model_summaries']
  summaries.to_csv('/content/gdrive/MyDrive/Project/lotte/data/{}_{}_performance.csv'.format(id,product))

  dates = f.export()['test_set_predictions']['DATE']
  real = f.export()['test_set_predictions']['actual']
  pred = f.export()['test_set_predictions']['arima3']
  pred_upper = f.export(cis = True)['test_set_predictions']['arima3_upperci']
  pred_lower = f.export(cis = True)['test_set_predictions']['arima3_lowerci']
  future_dates =  f.export()['all_fcsts']['DATE']
  future = f.export()['all_fcsts']['arima3']
  future_upper = f.export(cis = True)['all_fcsts']['arima3_upperci']
  future_lower = f.export(cis = True)['all_fcsts']['arima3_lowerci']

  raw_data = pd.DataFrame(MinMax_to_Raw(real, min_value, max_value), index = dates)
  pred_data = pd.DataFrame(MinMax_to_Raw(pred, min_value, max_value), index = dates)
  pred_upper_data = pd.DataFrame(MinMax_to_Raw(pred_upper, min_value, max_value), index = dates)
  pred_lower_data = pd.DataFrame(MinMax_to_Raw(pred_lower, min_value, max_value), index = dates)
  future_data = pd.DataFrame(MinMax_to_Raw(future, min_value, max_value), index = future_dates)
  future_upper_data = pd.DataFrame(MinMax_to_Raw(future_upper, min_value, max_value), index = future_dates)
  future_lower_data = pd.DataFrame(MinMax_to_Raw(future_lower, min_value, max_value), index = future_dates)

  dataset = pd.DataFrame(index = pd.date_range(data.index[0],future_dates[11],freq='D'), columns = ['raw','predicted','predicted_upperci','predicted_lowerci'])
  dataset.loc[data.index, 'raw'] = data['구매수량']
  dataset.loc[pred_data.index, 'predicted'] = pred_data[0].tolist()
  dataset.loc[future_data.index, 'predicted'] = future_data[0].tolist()
  dataset.loc[pred_upper_data.index, 'predicted_upperci'] = pred_upper_data[0].tolist()
  dataset.loc[future_upper_data.index, 'predicted_upperci'] = future_upper_data[0].tolist()
  dataset.loc[pred_lower_data.index, 'predicted_lowerci'] = pred_lower_data[0].tolist()
  dataset.loc[future_lower_data.index, 'predicted_lowerci'] = future_lower_data[0].tolist()
  dataset['predicted_upperci'] = dataset['predicted_upperci'].apply(lambda x : 0 if x<0 else x)
  dataset['predicted_lowerci'] = dataset['predicted_lowerci'].apply(lambda x : 0 if x<0 else x)

  plt.figure(figsize = (25, 8))
  plt.rc('xtick', labelsize=15)  # x축 눈금 폰트 크기 
  plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
  plt.rc('legend', fontsize=20)  # 범례 폰트 크기
  plt.plot(dataset['raw'], label = 'raw')
  plt.plot(dataset['predicted'], label = 'predicted')
  plt.fill_between(dataset[dataset.index>=dates.iloc[0]].index, dataset.loc[dataset.index>=dates.iloc[0],'predicted_lowerci'].tolist(), dataset.loc[dataset.index>=dates.iloc[0],'predicted_upperci'].tolist(), color = 'orange', alpha = 0.3)
  plt.legend()
  plt.savefig('/content/gdrive/MyDrive/Project/lotte/data/{} 고객의 {} 구매수량 예측'.format(id,product))

  plt.figure(figsize = (20, 8))
  plt.rc('xtick', labelsize=15)  # x축 눈금 폰트 크기 
  plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
  plt.rc('legend', fontsize=20)  # 범례 폰트 크기
  plt.plot(dataset.loc[dataset.index>dates.iloc[0],'raw'], label = 'raw')
  plt.plot(dataset['predicted'], label = 'predicted')
  plt.fill_between(dataset[dataset.index>=dates.iloc[0]].index, dataset.loc[dataset.index>=dates.iloc[0],'predicted_lowerci'].tolist(), dataset.loc[dataset.index>=dates.iloc[0],'predicted_upperci'].tolist(), color = 'orange', alpha = 0.3)
  plt.legend()
  plt.savefig('/content/gdrive/MyDrive/Project/lotte/data/{} 고객의 {} 구매수량 예측(예측범위만)'.format(id,product))

  dataset.to_csv('/content/gdrive/MyDrive/Project/lotte/data/{}_{}_raw_predict_values.csv'.format(id,product))